# Convert MIB file to hspy 

In [ ]:
%matplotlib qt
import hyperspy.api as hs #General hyperspy package
import pyxem as pxm #Electron diffraction tools based on hyperspy
import numpy as np #General numerical and matrix support
import matplotlib.pyplot as plt #Plotting tools
import matplotlib.colors as mcolors #Some plotting color tools

#Import path handling tool
from pathlib import Path
import json

def read_hdr(filename):
    """
    Return the content of a .hdr file as a dictionary
    """
    filename=Path(filename)
    if not filename.suffix == '.hdr':
        raise ValueError(f'Cannot read hdr file: File "{str(filename)}" is not a .hdr file')
    hdr = {}
    with filename.open('r') as f:
        for line in f.readlines():
            content = line.split(':', maxsplit=1)
            if len(content)>1:
                hdr[content[0].strip()] = content[1].strip()
    return hdr

## Load raw data

In [ ]:
datapath = Path(...)

In [ ]:
mib_data = pxm.load_mib(str(datapath))
hdr = read_hdr(datapath.with_suffix('.hdr'))

## Adjust metadata for the scan

In [ ]:
data = mib_data.data
scan_shape = (256,256) #Adjust to the size of your scan
chunksize = 32

data = mib_data.data
data = data.reshape(scan_shape + mib_data.axes_manager.signal_shape)
data = data.rechunk((chunksize,)*4)


In [ ]:
signal = pxm.signals.LazyElectronDiffraction2D(data)


In [ ]:
signal.metadata.General.title= '' #Set the title of the data  - this will appear over all plots


#Change these parameters to fit to your scan
signal.set_experimental_parameters(beam_energy=80,
                                   camera_length=10,
                                   scan_rotation=1,
                                   rocking_angle=1,
                                   rocking_frequency=10,
                                   exposure_time=10
                                   )

signal.set_scan_calibration(8.93) #Real space step size 


In [ ]:
def wavelength(V, 
            m0=9.1093837015*1e-31, 
            e=1.60217662*1e-19, 
            h=6.62607004*1e-34 , 
            c=299792458):
    """
    Return the wavelength of an accelerated electron in [m]
    
    Arguments
    ---------
    V : float, Acceleration voltage of electrons [kV]
    m0 : float, Rest mass of electron [kg]
    e : float, Elementary charge of electron [C]
    h : float, Planck' constant [m^2 kg/s]
    c : float, Speed of light in vacuum [m/s]
    """
    V = V*1E3
    return h / np.sqrt( 2 * m0 * e * V * ( 1.0 + ( e*V / ( 2*m0*c**2 ) ) ) )

def get_d_spacing(a = 1, b = 1, c = 1, h = 0, k = 0, l = 0, beta = np.pi/2, crystal = "monoclinic"):
    if crystal == "cubic":
        return 1/(h**2 / a**2 + k**2/b**2 + l**2/c**2)**0.5
    
    if crystal == "monoclinic":
        return np.sin(beta)/(h**2/a**2 + k**2 * np.sin(beta)**2/b**2 + l**2 / c**2 - 2*h*l*np.cos(beta)/a*c)**0.5

In [ ]:
l = wavelength(80)
p = 55E-6
c = 10E-2

signal.axes_manager[-1].scale = p/(l*c) * 1E-10
signal.axes_manager[-2].scale = p/(l*c) * 1E-10

signal.axes_manager[-1].units = "$Å^{-1}$"
signal.axes_manager[-2].units = "$Å^{-1}$"

signal.axes_manager[-1].Name = "kx"
signal.axes_manager[-2].Name = "ky"
print(signal.axes_manager)

## Save the signal 

In [ ]:
signal.save(str(datapath.parent) + r"\sped.hspy")